In [1]:
import pandas as pd
import re

In [3]:
nhanesInterviewData = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DS1IDS_I.XPT", format = 'XPORT', encoding='iso-8859-1')
nhanesProductData = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/1999-2000/DSPI.XPT", format = 'XPORT', encoding='iso-8859-1')
nhanesIngredientInfo = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/1999-2000/DSII.XPT", format= 'XPORT', encoding = 'iso-8859-1')

In [4]:
nhanesInterviewData.columns

Index(['SEQN', 'WTDRD1', 'WTDR2D', 'DR1DRSTZ', 'DR1EXMER', 'DRDINT', 'DR1DBIH',
       'DR1DAY', 'DR1LANG', 'DS1LOC', 'DSDSUPID', 'DSDSUPP', 'DS1MTCH',
       'DS1ANTA', 'DS1ACTSS', 'DS1IKCAL', 'DS1IPROT', 'DS1ICARB', 'DS1ISUGR',
       'DS1IFIBE', 'DS1ITFAT', 'DS1ISFAT', 'DS1IMFAT', 'DS1IPFAT', 'DS1ICHOL',
       'DS1ILYCO', 'DS1ILZ', 'DS1IVB1', 'DS1IVB2', 'DS1INIAC', 'DS1IVB6',
       'DS1IFA', 'DS1IFDFE', 'DS1ICHL', 'DS1IVB12', 'DS1IVC', 'DS1IVK',
       'DS1IVD', 'DS1ICALC', 'DS1IPHOS', 'DS1IMAGN', 'DS1IIRON', 'DS1IZINC',
       'DS1ICOPP', 'DS1ISODI', 'DS1IPOTA', 'DS1ISELE', 'DS1ICAFF', 'DS1IIODI'],
      dtype='object')

In [5]:
nhanesProductData.columns

Index(['DSDPID', 'DSDGENRC', 'DSDSUPP', 'DSDSRCE', 'DSDTYPE', 'DSDSERVQ',
       'DSDSERVU', 'DSDCNTV', 'DSDCNTM', 'DSDCNTA', 'DSDCNTB', 'DSDCNTO',
       'DSDSUPID'],
      dtype='object')

### The column 'DSDSUPID' indicates the supplementID. This can be used for joining the interview data with the product and ingredient information

In [ ]:
## List of Supplement IDs where the supplement name contains 'cranberry'

In [6]:
cranberrySuppID = nhanesProductData['DSDSUPID'][nhanesProductData['DSDSUPP'].str.contains('CRANBERRY')]

In [8]:
cranberrySuppID.shape  ## Total of 112 unique supplements

(112,)

In [9]:
## Search for this list in the 1st day interview data

In [10]:
interviewCranberryProd = nhanesInterviewData[nhanesInterviewData['DSDSUPID'].isin(cranberrySuppID)]

In [12]:
interviewCranberryProd.shape  ## total of 27 observations (could be same product repeated multiple times) in 2017-18 which have a cranberry in their name

(27, 49)

## Product info about these 27 products

In [13]:
cranberry17ProdInfo = nhanesProductData[nhanesProductData['DSDSUPID'].isin(interviewCranberryProd['DSDSUPID'])]

In [14]:
cranberry17ProdInfo.shape  ## total of 14 unique cranberry supplements

(14, 13)

In [151]:
cranberry17ProdInfo[['DSDSUPID','DSDSUPP']]  ## Generally, the first 2 words are the brand name

,DSDSUPID,DSDSUPP
4586,1888214002,DEFAULT CRANBERRY
9594,1000548901,TRUNATURE CRANBERRY ANTIOXIDANT ACTIVITY WITH ...
9888,1000959100,NATURE'S BOUNTY CRANBERRY FRUIT 4200 MG EQUIVA...
12458,1001153800,"AZO CRANBERRY 25,000 MG CRANBERRY FRUIT EQUIVA..."
12514,1001155000,SPRING VALLEY STANDARDIZED EXTRACT CRANBERRY 5...
12606,1001155600,SPRING VALLEY STANDARDIZED EXTRACT CRANBERRY 5...
13157,1000924401,PURITAN'S PRIDE CRANBERRY FRUIT CONCENTRATE 42...
13715,1001227500,THERALOGIX THERACRAN HP CRANBERRY STANDARDIZED...
13766,1001066301,CVS HEALTH CRANBERRY 4200 MG EQUIVALENT PER SO...
13771,1001231500,PURITAN'S PRIDE NATURAL CRANBERRY FRUIT CONCEN...


In [126]:
cranberry17ProdInfo['BRAND_NAME'] = cranberry17ProdInfo['DSDSUPP'].str.replace("'", "")

In [128]:
cranberry17ProdInfo['BRAND_NAME'] = cranberry17ProdInfo['BRAND_NAME'].str.extract("(\\w+\\s\\w+)")

In [129]:
cranberry17ProdInfo['BRAND_NAME']

4586        DEFAULT CRANBERRY
9594      TRUNATURE CRANBERRY
9888           NATURES BOUNTY
12458           AZO CRANBERRY
12514           SPRING VALLEY
12606           SPRING VALLEY
13157          PURITANS PRIDE
13715    THERALOGIX THERACRAN
13766              CVS HEALTH
13771          PURITANS PRIDE
13814           SPRING VALLEY
13941               GARDEN OF
14443      CLEARTRACT URINARY
14471           CRANBERRY 500
Name: BRAND_NAME, dtype: object

In [130]:
cranberry17ProdInfo.columns

Index(['DSDPID', 'DSDGENRC', 'DSDSUPP', 'DSDSRCE', 'DSDTYPE', 'DSDSERVQ',
       'DSDSERVU', 'DSDCNTV', 'DSDCNTM', 'DSDCNTA', 'DSDCNTB', 'DSDCNTO',
       'DSDSUPID', 'BRAND_NAME'],
      dtype='object')

In [132]:
cranberry17ProdInfo['DSDSUPID'] = cranberry17ProdInfo['DSDSUPID'].astype('int64')
interviewCranberryProd['DSDSUPID'] = interviewCranberryProd['DSDSUPID'].astype('int64')

In [170]:
cranberry17InterviewProd = cranberry17ProdInfo.join(interviewCranberryProd.filter(['SEQN', 'WTDRD1', 'DSDSUPID']).set_index('DSDSUPID'), on='DSDSUPID', how = 'inner', lsuffix='prod')

In [172]:
cranberry17InterviewProd.head()

,DSDPID,DSDGENRC,DSDSUPP,DSDSRCE,DSDTYPE,DSDSERVQ,DSDSERVU,DSDCNTV,DSDCNTM,DSDCNTA,DSDCNTB,DSDCNTO,DSDSUPID,BRAND_NAME,SEQN,WTDRD1
4586,6386.0,NaN,DEFAULT CRANBERRY,NaN,4.0,1.0,18.0,5.397605e-79,5.397605e-79,5.397605e-79,1.0,5.397605e-79,1888214002,DEFAULT CRANBERRY,84125.0,70444.394060
4586,6386.0,NaN,DEFAULT CRANBERRY,NaN,4.0,1.0,18.0,5.397605e-79,5.397605e-79,5.397605e-79,1.0,5.397605e-79,1888214002,DEFAULT CRANBERRY,87511.0,204758.713565
4586,6386.0,NaN,DEFAULT CRANBERRY,NaN,4.0,1.0,18.0,5.397605e-79,5.397605e-79,5.397605e-79,1.0,5.397605e-79,1888214002,DEFAULT CRANBERRY,88731.0,15634.728495
4586,6386.0,NaN,DEFAULT CRANBERRY,NaN,4.0,1.0,18.0,5.397605e-79,5.397605e-79,5.397605e-79,1.0,5.397605e-79,1888214002,DEFAULT CRANBERRY,90908.0,10041.566279
4586,6386.0,NaN,DEFAULT CRANBERRY,NaN,4.0,1.0,18.0,5.397605e-79,5.397605e-79,5.397605e-79,1.0,5.397605e-79,1888214002,DEFAULT CRANBERRY,90922.0,19227.490463


In [153]:
cranberry17InterviewProd[cranberry17InterviewProd['DSDSUPID'] == 1000548901]

,DSDPID,DSDGENRC,DSDSUPP,DSDSRCE,DSDTYPE,DSDSERVQ,DSDSERVU,DSDCNTV,DSDCNTM,DSDCNTA,DSDCNTB,DSDCNTO,DSDSUPIDprod,BRAND_NAME,SEQN,WTDRD1,DSDSUPID
